# Requires caption and gaze behavior file to determine turn taking and turn keeping seuqences

In [3]:
import numpy as np

import pandas as pd

import re
#import srt
import time
from datetime import datetime, timedelta
back_channel=['yeah','oh','um','okay','cool','right','uh']

In [9]:
all_date=["12-15-2021","01-28-2022","02-11-2022","03-04-2022","03-05-2022","03-11-2022","03-12-2022"]
date=all_date[5]
all_session=['1','2','3']
all_sessions=[['1','2','3','4'],['1','2','3','4','5','6'],['2','3'],['1','2','3'],['1','2']]

In [7]:
date

'03-11-2022'

In [10]:
for idd, all_session in enumerate(all_sessions):
    date=all_date[idd]
    for session in all_session:

        p1="../3people/"+date+"/Caption/Session_"+session+"_PC_1_sentence.csv"
        p2="../3people/"+date+"/Caption/Session_"+session+"_PC_2_sentence.csv"
        p3="../3people/"+date+"/Caption/Session_"+session+"_PC_3_sentence.csv"
        p1_audio = pd.read_csv(p1)
        p2_audio = pd.read_csv(p2)
        p3_audio = pd.read_csv(p3)
        data="../3people/"+date+"/Session_"+session+"_gazeBehavior.txt"
        gaze = pd.read_csv(data,sep='\t')
        audio = pd.DataFrame(0, index=np.arange(len(gaze)), columns=['P1 audio', 'P2 audio', 'P3 audio'])
        audio['P1_SID']='0'
        audio['P2_SID']='0'
        audio['P3_SID']='0'
        for index, row in p1_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            if start_frame < 600:
                p1_audio = p1_audio.drop(index)
        p1_audio.reset_index()

        for index, row in p2_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            if start_frame < 600:
                p2_audio = p2_audio.drop(index)
        p2_audio.reset_index()

        for index, row in p3_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            if start_frame < 600:
                p3_audio = p3_audio.drop(index)
        p3_audio.reset_index()
        for index, row in p1_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            end = datetime.strptime(row['end'], '%H:%M:%S.%f')
            end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
            SID=row['SID']
            ### Determine if the sentence is music
            is_music=0
            if row['Sentence']=='[Music]':
                is_music=1

            ### Determine if the sentence is back-channeling
            sentence_back=1
            for word in row['Sentence'].split():
                word_back=0
                for back in back_channel:
                    if word==back:
                        word_back=1
                if word_back==0:
                    sentence_back=0

            ### Print out the audio        
            if is_music==0:        
                if sentence_back==1:
                    ### The whole sentence is back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P1 audio'][i]=2
                        audio['P1_SID'][i]=SID
                else:
                    ### The sentence is not back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P1 audio'][i]=1
                        audio['P1_SID'][i]=SID
        for index, row in p2_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            end = datetime.strptime(row['end'], '%H:%M:%S.%f')
            end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
            SID=row['SID']
            ### Determine if the sentence is music
            is_music=0
            if row['Sentence']=='[Music]':
                is_music=1

            ### Determine if the sentence is back-channeling
            sentence_back=1
            for word in row['Sentence'].split():
                word_back=0
                for back in back_channel:
                    if word==back:
                        word_back=1
                if word_back==0:
                    sentence_back=0

            ### Print out the audio        
            if is_music==0:        
                if sentence_back==1:
                    ### The whole sentence is back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P2 audio'][i]=2
                        audio['P2_SID'][i]=SID
                else:
                    ### The sentence is not back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P2 audio'][i]=1
                        audio['P2_SID'][i]=SID
        for index, row in p3_audio.iterrows():
            start = datetime.strptime(row['start'], '%H:%M:%S.%f')
            start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
            end = datetime.strptime(row['end'], '%H:%M:%S.%f')
            end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
            SID=row['SID']
            ### Determine if the sentence is music
            is_music=0
            if row['Sentence']=='[Music]':
                is_music=1

            ### Determine if the sentence is back-channeling
            sentence_back=1
            for word in row['Sentence'].split():
                word_back=0
                for back in back_channel:
                    if word==back:
                        word_back=1
                if word_back==0:
                    sentence_back=0

            ### Print out the audio        
            if is_music==0:        
                if sentence_back==1:
                    ### The whole sentence is back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P3 audio'][i]=2
                        audio['P3_SID'][i]=SID
                else:
                    ### The sentence is not back channeling
                    for i in range(start_frame-1,end_frame):
                        audio['P3 audio'][i]=1
                        audio['P3_SID'][i]=SID
        final = pd.concat([audio, gaze],
                          axis = 1)
        new_col=['P1 audio', 'P2 audio', 'P3 audio','P1 Gaze', 'P2 Gaze', 'P3 Gaze','P1_SID','P2_SID','P3_SID']
        final=final[new_col]
        final.to_csv("../3people/Training/AudioAndGaze/"+date+"_Session_"+session+"_audio_gaze.csv",index=False)



C:\Users\DreamFall\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DreamFall\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DreamFall\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DreamFall\anaconda3\envs\pytorch\lib\sit

In [263]:
# date="03-12-2022"
# session="1"

### Read caption file

In [264]:
# p1="../3people/"+date+"/Caption/Session_"+session+"_PC_1_sentence.csv"
# p2="../3people/"+date+"/Caption/Session_"+session+"_PC_2_sentence.csv"
# p3="../3people/"+date+"/Caption/Session_"+session+"_PC_3_sentence.csv"
# p1_audio = pd.read_csv(p1)
# p2_audio = pd.read_csv(p2)
# p3_audio = pd.read_csv(p3)

### Read Gaze Behavior

In [265]:
# data="../3people/"+date+"/Session_"+session+"_gazeBehavior.txt"
# gaze = pd.read_csv(data,sep='\t')

In [266]:
# len(gaze)

### Record speaking time: 0 means silent, 1 means speaking

In [267]:
# audio = pd.DataFrame(0, index=np.arange(len(gaze)), columns=['P1 audio', 'P2 audio', 'P3 audio'])
# audio['SID']='0'

In [1]:
# for index, row in p1_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     if start_frame < 600:
#         p1_audio = p1_audio.drop(index)
# p1_audio.reset_index()

# for index, row in p2_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     if start_frame < 600:
#         p2_audio = p2_audio.drop(index)
# p2_audio.reset_index()

# for index, row in p3_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     if start_frame < 600:
#         p3_audio = p3_audio.drop(index)
# p3_audio.reset_index()

NameError: name 'p1_audio' is not defined

In [269]:
# for index, row in p1_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     SID=row['SID']
#     ### Determine if the sentence is music
#     is_music=0
#     if row['Sentence']=='[Music]':
#         is_music=1
    
#     ### Determine if the sentence is back-channeling
#     sentence_back=1
#     for word in row['Sentence'].split():
#         word_back=0
#         for back in back_channel:
#             if word==back:
#                 word_back=1
#         if word_back==0:
#             sentence_back=0
            
#     ### Print out the audio        
#     if is_music==0:        
#         if sentence_back==1:
#             ### The whole sentence is back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P1 audio'][i]=2
#                 audio['SID'][i]=SID
#         else:
#             ### The sentence is not back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P1 audio'][i]=1
#                 audio['SID'][i]=SID

In [270]:
# for index, row in p2_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     SID=row['SID']
#     ### Determine if the sentence is music
#     is_music=0
#     if row['Sentence']=='[Music]':
#         is_music=1
    
#     ### Determine if the sentence is back-channeling
#     sentence_back=1
#     for word in row['Sentence'].split():
#         word_back=0
#         for back in back_channel:
#             if word==back:
#                 word_back=1
#         if word_back==0:
#             sentence_back=0
            
#     ### Print out the audio        
#     if is_music==0:        
#         if sentence_back==1:
#             ### The whole sentence is back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P2 audio'][i]=2
#                 audio['SID'][i]=SID
#         else:
#             ### The sentence is not back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P2 audio'][i]=1
#                 audio['SID'][i]=SID

In [271]:
# for index, row in p3_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     SID=row['SID']
#     ### Determine if the sentence is music
#     is_music=0
#     if row['Sentence']=='[Music]':
#         is_music=1
    
#     ### Determine if the sentence is back-channeling
#     sentence_back=1
#     for word in row['Sentence'].split():
#         word_back=0
#         for back in back_channel:
#             if word==back:
#                 word_back=1
#         if word_back==0:
#             sentence_back=0
            
#     ### Print out the audio        
#     if is_music==0:        
#         if sentence_back==1:
#             ### The whole sentence is back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P3 audio'][i]=2
#                 audio['SID'][i]=SID
#         else:
#             ### The sentence is not back channeling
#             for i in range(start_frame-1,end_frame):
#                 audio['P3 audio'][i]=1
#                 audio['SID'][i]=SID

In [272]:
#for index, row in p3_audio.iterrows():
#    start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#    start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#    end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#    end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#    for i in range(start_frame-1,end_frame):
#        audio['P3 audio'][i]=1

In [273]:
#audio.to_csv("test.csv",index=False)

In [274]:
# final = pd.concat([audio, gaze],
#                   axis = 1)
# new_col=['P1 audio', 'P2 audio', 'P3 audio','P1 Gaze', 'P2 Gaze', 'P3 Gaze','SID']
# final=final[new_col]

In [275]:
# new_col=['P1 audio', 'P2 audio', 'P3 audio','P1 Gaze', 'P2 Gaze', 'P3 Gaze','SID']
# final=final[new_col]

In [276]:
# final

### column 1-3 is speaker's speaking status, 1 means speaking, 0 means silent, 2 means back-channeling
### column 4-6 is gaze behavior, 1 means looking left, 2 means looking right, 0 means looking nothing

In [277]:
# final.to_csv("../3people/Training/AudioAndGaze/"+date+"_Session_"+session+"_audio_gaze.csv",index=False)

### Determine the sequence is turn-taking or turn-keeping

In [278]:
# p1_taking=[]
# p1_keeping=[]
# p2_taking=[]
# p2_keeping=[]
# p3_taking=[]
# p3_keeping=[]

In [279]:
# for index, row in p1_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     find=end_frame

#     while True:
#         if find>=len(final):
#             break
#         elif audio['P1 audio'][find]==1 or audio['P2 audio'][find]==1 or audio['P3 audio'][find]==1:
#             if audio['P1 audio'][find]==1:
#                 p1_keeping.append([start_frame-1,end_frame])
#             else:
#                 p1_taking.append([start_frame-1,end_frame])
#             break
#         find = find+1
# for index, row in p2_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     find=end_frame

#     while True:
#         if find>=len(final):
#             break
#         elif audio['P1 audio'][find]==1 or audio['P2 audio'][find]==1 or audio['P3 audio'][find]==1:
#             if audio['P2 audio'][find]==1:
#                 p2_keeping.append([start_frame-1,end_frame])
#             else:
#                 p2_taking.append([start_frame-1,end_frame])
#             break
#         find = find+1
# for index, row in p3_audio.iterrows():
#     start = datetime.strptime(row['start'], '%H:%M:%S.%f')
#     start_frame=round((start.minute*60+start.second+start.microsecond/1000000)*60)
#     end = datetime.strptime(row['end'], '%H:%M:%S.%f')
#     end_frame=round((end.minute*60+end.second+end.microsecond/1000000)*60)
#     find=end_frame

#     while True:
#         if find>=len(final):
#             break
#         elif audio['P1 audio'][find]==1 or audio['P2 audio'][find]==1 or audio['P3 audio'][find]==1:
#             if audio['P3 audio'][find]==1:
#                 p3_keeping.append([start_frame-1,end_frame])
#             else:
#                 p3_taking.append([start_frame-1,end_frame])
#             break
#         find = find+1

In [280]:
# p2_audio

In [281]:
# len(audio)

### Statistics about our data

In [282]:
# taking=len(p1_taking)+len(p2_taking)+len(p3_taking)
# print("Amount of all turn-taking sequences:"+str(taking))
# keeping=len(p1_keeping)+len(p2_keeping)+len(p3_keeping)
# print("Amount of all turn-keeping sequences:"+str(keeping))
# print("The portion of turn-taking and turn-keeping is "+str(round(taking/keeping, 1))+":1")

### Export intervals
* column 1: start
* column 2 end
* column 3 who's speaking
* column 4 state: 0 means keeping, 1 means turning

In [283]:
# intervals = pd.DataFrame(columns=['start','end','person','state'])
# for interval in p1_keeping:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [1], 'state':[0]})
#     intervals = pd.concat([intervals,temp])
# for interval in p2_keeping:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [2], 'state':[0]})
#     intervals = pd.concat([intervals,temp])
# for interval in p3_keeping:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [3], 'state':[0]})
#     intervals = pd.concat([intervals,temp])
# for interval in p1_taking:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [1], 'state':[1]})
#     intervals = pd.concat([intervals,temp])
# for interval in p2_taking:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [2], 'state':[1]})
#     intervals = pd.concat([intervals,temp])
# for interval in p3_taking:
#     temp = pd.DataFrame({'start': [interval[0]], 'end': [interval[1]], 'person': [3], 'state':[1]})
#     intervals = pd.concat([intervals,temp])
# #intervals.reset_index(drop=True)

In [284]:
# p1_keeping

In [285]:
# intervals.to_csv("../3people/Training/Intervals/"+date+"_Session_"+session+"_intervals.csv",index=False)